# OLS4 API Search Example

This notebook demonstrates searching the EBI OLS4 (Ontology Lookup Service) API.

In [1]:
import requests
import json
import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables from .env file
env_path = Path("..") / ".env"
load_dotenv(dotenv_path=env_path)

# Get BioPortal API key
BIOPORTAL_API_KEY = os.getenv("BIOPORTAL_API_KEY")

if not BIOPORTAL_API_KEY:
    print("⚠️  Warning: BIOPORTAL_API_KEY not set in .env file")
    print("   Get your free API key at: https://bioportal.bioontology.org/accounts/new")
else:
    print("✓ BioPortal API key loaded")

✓ BioPortal API key loaded


## Search OLS4

In [2]:
base_url = "https://www.ebi.ac.uk/ols4/api/search"
params = {
    "q": "thermophile",
    "type": "class"
}

response = requests.get(base_url, params=params)

if response.status_code == 200:
    search_results = response.json()
    # Display the search results in a readable format
    print(json.dumps(search_results, indent=4))
else:
    print(f"Error: {response.status_code}")
    print(response.text)

{
    "response": {
        "docs": [
            {
                "iri": "http://purl.obolibrary.org/obo/OMP_0005004",
                "ontology_name": "omp",
                "ontology_prefix": "OMP",
                "short_form": "OMP_0005004",
                "description": [
                    "A population growth phenotype where growth of a microbial population can occur at 55 C or higher and optimal growth occurs between 55 C and 65 C."
                ],
                "label": "thermophile",
                "obo_id": "OMP:0005004",
                "type": "class"
            },
            {
                "iri": "http://purl.obolibrary.org/obo/MICRO_0001537",
                "ontology_name": "micro",
                "ontology_prefix": "MICRO",
                "short_form": "MICRO_0001537",
                "description": [],
                "label": "thermophile",
                "obo_id": "MICRO:0001537",
                "type": "class"
            },
            {
       

## Search BioPortal

In [3]:
bioportal_url = "https://data.bioontology.org/search"
bioportal_params = {
    "q": "thermophile",
    # Uncomment to filter to specific ontologies:
    # "ontologies": "OMP,PATO,MCO",
    "require_definitions": "false"  # Changed to false to get more results
}

# Add API key to headers
headers = {}
if BIOPORTAL_API_KEY:
    headers["Authorization"] = f"apikey token={BIOPORTAL_API_KEY}"
    print(f"✓ Using BioPortal API key")
else:
    print("❌ ERROR: BioPortal API key not set - request will fail with 401")
    print("   Add BIOPORTAL_API_KEY to .env file")
    print("   Get free key at: https://bioportal.bioontology.org/accounts/new")

response = requests.get(bioportal_url, params=bioportal_params, headers=headers)

if response.status_code == 200:
    bioportal_results = response.json()
    total = bioportal_results.get('totalCount', 0)
    page_count = bioportal_results.get('pageCount', 0)
    collection = bioportal_results.get('collection', [])
    print(f"\n✓ Found {total} total results ({page_count} on this page)")
    print(f"Showing {len(collection)} results:\n")
    
    # Show summary of results
    for result in collection[:5]:  # Show first 5
        print(f"  - {result.get('prefLabel', 'N/A')}")
        print(f"    Ontology: {result.get('links', {}).get('ontology', 'N/A')}")
        print(f"    IRI: {result.get('@id', 'N/A')[:80]}...")
        print()
    
    if len(collection) > 5:
        print(f"  ... and {len(collection) - 5} more")
else:
    print(f"\n❌ Error: {response.status_code}")
    if response.status_code == 401:
        print("   Authentication failed - check your BIOPORTAL_API_KEY in .env")
    print(f"   Response: {response.text[:200]}")

✓ Using BioPortal API key

✓ Found 18 total results (1 on this page)
Showing 18 results:

  - Thermophile
    Ontology: https://data.bioontology.org/ontologies/NATPRO
    IRI: http://www.owl-ontologies.com/NPOntology.owl#Thermophile...

  - thermophile
    Ontology: https://data.bioontology.org/ontologies/OCHV
    IRI: http://sbmi.uth.tmc.edu/ontology/ochv#C0597570...

  - thermophile
    Ontology: https://data.bioontology.org/ontologies/OMP
    IRI: http://purl.obolibrary.org/obo/OMP_0005004...

  - thermophile
    Ontology: https://data.bioontology.org/ontologies/IOBC
    IRI: http://purl.jp/bio/4/id/200906071075666552...

  - Chaetomium thermophile
    Ontology: https://data.bioontology.org/ontologies/SNOMEDCT
    IRI: http://purl.bioontology.org/ontology/SNOMEDCT/54003002...

  ... and 13 more
